In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
from matplotlib import colors
import networkx as nx
from pyvis.network import Network


In [2]:
PATH = "../serial-reproduction-with-selection/analysis/data/rivka-necklace-rep-data/psynet/data/"
nodes = pd.read_csv(PATH + "node.csv", low_memory=False)
networks = pd.read_csv(PATH + "network.csv", low_memory=False)

In [3]:
# filter networks
network_data = networks
network_data = network_data[network_data["role"] == "experiment"]
network_data = network_data[network_data["failed"] == 'f']
network_data = network_data[network_data["trial_maker_id"] == 'graph_experiment'] # To update

experiment_net_id = list(network_data['id'].to_numpy())

# filter nodes
node_data = nodes
node_data = node_data[nodes["type"] == "graph_chain_node"]
node_data = node_data[node_data["failed"] == "f"]
node_data = node_data[node_data["network_id"].isin(experiment_net_id)]
node_data = node_data[["id", "network_id", "degree", "definition", "seed", "vertex_id", "dependent_vertex_ids"]]
node_data = node_data.sort_values(["network_id", "degree"])

# read infos, filter like nodes
infos = pd.read_csv(PATH + "info.csv")
info_data = infos
info_data = info_data[infos["type"] == "graph_chain_trial"]
info_data = info_data[info_data["failed"] == "f"]
info_data = info_data[["id", "creation_time", "details", "origin_id", "network_id", "participant_id"]] # TODO: Probably want more columns here 

info_data = info_data.sort_values(["network_id", "origin_id"])

In [8]:

def generate_graph(degree):
    ''' Given a degree (int or float), return a Graph containing nodes (with metadata from infos) and edges in that degree. 
    '''
    # validation: ensure degree is a float 
    if not isinstance(degree, float):
        try:
            degree = float(degree)
        except Exception as ex:
            raise("When converting degree to float, the following exception occured: " + str(ex))

    # create graph 
    G = nx.Graph()

    # add nodes from node_data to the Graph
    # nodes are identified by their node_id
    # nodes have named attributes vertex_id and degree
    deg_nodes = node_data[node_data["degree"] == degree]
    for node_id in deg_nodes["id"].values.tolist():
        # extract vertex id 
        vert_id = deg_nodes[deg_nodes["id"] == node_id]["vertex_id"].values[0]

        # add metadata from infos 
        info_row = info_data[info_data["origin_id"] == node_id]
        try:
            creation_time = info_row["creation_time"].values[0]
        except:
            creation_time = None

        G.add_node(node_id, vertex_id=vert_id, degree=degree, creation_time=creation_time)

    # add edges to the Graph: iterate over deg_nodes, add incoming edges
    # using dependent_vertex_ids column 
    edges = {}
    for i, ser in deg_nodes.iterrows():
        node_id = ser["id"]

        # get dependent vertices (incoming edges) in a list 
        dependent_vertices = ser["dependent_vertex_ids"].strip('][').split(',')

        # find the corresponding row of deg_nodes for each vertex_id
        dependent_nodes = [deg_nodes[deg_nodes["vertex_id"] == float(v)] for v in dependent_vertices]

        # extract the node_id from each dependent vertex row
        dependent_nodes = [n["id"].values[0] for n in dependent_nodes]

        # add as edges (dependent node --> curent node)
        edge_list = [(int(dependent_node), int(node_id)) for dependent_node in dependent_nodes]
        G.add_edges_from(edge_list)

    return G

In [9]:
for d in range(int(node_data["degree"].max())):
    # convert to PyVis and show
    pyvis_net = Network('500px', '500px')
    pyvis_net.from_nx(generate_graph(float(d+1)))
    print('generated graph of degree ' + str(float(d+1)))

    # add labels
    for n in pyvis_net.nodes:
        n['label'] = str(int(n['vertex_id'])) # gets changed to a float if you try to convert it in networkx for some reason
        n['title'] = str(n['creation_time'])

    pyvis_net.show('slice_%d.html' % d)

generated graph of degree 1.0
generated graph of degree 2.0
generated graph of degree 3.0
generated graph of degree 4.0
generated graph of degree 5.0
generated graph of degree 6.0
generated graph of degree 7.0
generated graph of degree 8.0
generated graph of degree 9.0
generated graph of degree 10.0
generated graph of degree 11.0
generated graph of degree 12.0
generated graph of degree 13.0
generated graph of degree 14.0
generated graph of degree 15.0
generated graph of degree 16.0
generated graph of degree 17.0
generated graph of degree 18.0
generated graph of degree 19.0
generated graph of degree 20.0
generated graph of degree 21.0


In [15]:
slice_graphs = []
min_deg = int(node_data["degree"].min())
max_deg = int(node_data["degree"].max())    # assumes degrees are continuous

for d in range(min_deg, max_deg + 1):
    d = float(d)
    G = nx.DiGraph()

    # add nodes to new Graph
    deg = node_data[node_data["degree"] == d]
    for node_id in deg["id"].values.tolist():
        v_id = deg[deg["id"] == node_id]["vertex_id"].values[0]
        G.add_node(node_id, vertex_id=v_id, degree=d)

    # add edges to the Graph 
    edges = {}
    for i, ser in deg.iterrows():
        node_id = ser["id"]

        # get dependent nodes 
        dependent_vertices = ser["dependent_vertex_ids"].strip('][').split(',')
        dependent_nodes = [deg[deg["vertex_id"] == float(v)] for v in dependent_vertices]
        dependent_nodes = [n["id"].values[0] for n in dependent_nodes]

        # add as edges
        edge_list = [(int(dependent_node), int(node_id)) for dependent_node in dependent_nodes]
        G.add_edges_from(edge_list)

    # Add metadata from infos to each node
    for n in G.nodes.items():
        n_id = n[0]
        row = info_data[info_data["origin_id"] == n_id]
        G.nodes[n_id]['creation_time'] = row["creation_time"].values[0]
        # TODO: Add other infos here 

    slice_graphs.append(G)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
for d in range(max_deg-min_deg):
    # convert to PyVis and show
    pyvis_net = Network('500px', '500px')
    pyvis_net.from_nx(slice_graphs[d])

    # add labels
    for n in pyvis_net.nodes:
        n['label'] = str(int(n['vertex_id'])) # gets changed to a float if you try to convert it in networkx for some reason
        n['title'] = str(n['degree'])

    pyvis_net.show('slice_%d.html' % d)